In [2]:
# Core data manipulation and analysis libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations and arrays

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import random

In [4]:
class CorrEncoder:
    """
    CorrEncoder: Takes a dataset as input and uses it for the encode function. Encodes the filtered categories then draws correlations.
    If correlation is above the threshold adds it to a new dataframe then returns the one hot encoded values with the labels.

    Initialisation:
        - data (pd.DataFrame): The Dataset that contains the target column and target label variables.
    """
    
    def __init__(self, data):
        self.data = data.copy()
        # Removes Label for the multi-class processing as it is based on the label category (threat or not).
        self.data = self.data.drop(columns=['attack_cat'])

    def encode(self, target_column, sparse_n, threshold, print_data):
        """
        encode: Takes a target column and target label to encode and draw correlations from. The target column is iterated through
        for all categories that contain more positive values than defined in sparse_n. This allows for filtering of sparse categories.
        The function then one hot encodes the given category with the static target column and draws correlations for them. If correlation
        is greater then threshold then add it to the new DataFrame. The function returns the one hot encoded categories that pass the
        threshold with the target label.

        The purpose of this function is to resolve the high cardinality problem in one hot encoding.

        Parameters:
            - target_column (string): The name of the target column. The target column should contain the various categories to encode.
            - sparse_n (integer): The minimum amount of positive values required for a category after encoding (deals with sparse categories).
            - threshold (float): The threshold for correlation. The function creates onehot encoded columns of all variables that have correlation
              higher that the threshold to the target label.

        Returns:
            - ohe_df (pd.DataFrame): The one hot encoded values from the target column.
        """

        self.data[target_column] = self.data[target_column].astype(str)
        value_counts = self.data[target_column].value_counts()
        # Check if number of 1s is above the given threshold set by sparse_n.
        categories = value_counts[value_counts > sparse_n].index.tolist()
        ohe_list = []
        # Attack category for multi-class binary.
        attack_cat = self.data['label']
        
        # Go through each unique category in the target column.
        for c in categories:
            col_name = f'{target_column}_{c}'

            # Create the binary encoding column for the current category and target label.
            corr_column = (self.data[target_column] == c).astype(int)
            correlation = corr_column.corr(attack_cat)

            # Check if absolute correlation is greater than threshold.
            if abs(correlation) > threshold:
                corr_column.name = col_name
                ohe_list.append(corr_column)
        if print_data:
            print('Number of Encoded Features for', target_column)
            print(len(ohe_list))
        if ohe_list:
            # NOTE: This section can be expanded to include print outs but at the moment am focusing on the evaluations.
            ohe_df = pd.concat(ohe_list, axis=1)
            return ohe_df
        else:
            return pd.DataFrame()

In [11]:
df = pd.read_csv('Cleaned_full_data.csv')
print(len(df))
df['attack_cat'] = df['attack_cat'].str.replace('Backdoors', 'Backdoor')
# Interesting here. I was getting this error when encoding but you seemed to have fixed it.
df = df[~df['sport'].astype(str).str.startswith('0x')]
df = df[~df['dsport'].astype(str).str.startswith('0x')]
print(len(df))

2540047
2539744


In [13]:
threshold = 0.1
encoder = CorrEncoder(df)
ohe1 = encoder.encode('state', 30, threshold, True)
ohe2 = encoder.encode('service', 30, threshold, True)
ohe3 = encoder.encode('proto', 30, threshold, True)
ohe4 = encoder.encode('sport', 30, threshold, True)
ohe5 = encoder.encode('srcip', 30, threshold, True)
ohe6= encoder.encode('dstip', 30, threshold, True)

Number of Encoded Features for state
3
Number of Encoded Features for service
2
Number of Encoded Features for proto
3
Number of Encoded Features for sport
3
Number of Encoded Features for srcip
14
Number of Encoded Features for dstip
19


In [14]:
threshold = 0.01
encoder = CorrEncoder(df)
ohe1 = encoder.encode('state', 30, threshold, True)
ohe2 = encoder.encode('service', 30, threshold, True)
ohe3 = encoder.encode('proto', 30, threshold, True)
ohe4 = encoder.encode('sport', 30, threshold, True)
ohe5 = encoder.encode('srcip', 30, threshold, True)
ohe6= encoder.encode('dstip', 30, threshold, True)

Number of Encoded Features for state
3
Number of Encoded Features for service
11
Number of Encoded Features for proto
130


KeyboardInterrupt: 

In [ ]:
threshold = 0.001
encoder = CorrEncoder(df)
ohe1 = encoder.encode('state', 30, threshold, True)
ohe2 = encoder.encode('service', 30, threshold, True)
ohe3 = encoder.encode('proto', 30, threshold, True)
ohe4 = encoder.encode('sport', 30, threshold, True)
ohe5 = encoder.encode('srcip', 30, threshold, True)
ohe6= encoder.encode('dstip', 30, threshold, True)